# Assignment 5 answer

## 1. TreeNode [3pt]
Has four properties including key, distance, left and right, where key is for storing species name,
distance is for branch length, left is for the left child and right is for the right child. The class has
a function print to print the tree structure in the form of (key, distance, left, right). For example,
(‘W_X’,0.5,(‘W’,0,(),()),(‘X’,0,(),())) is for the tree W_X with left child (‘W’,0,(),()) and right child
(‘X’,0,(),()); the branch length to its child is 0.5

In [8]:
class TreeNode:
    def __init__(self, name = "", distance = -1):
        self.name = name
        self.distance = float(distance)
        self.left = None
        self.right = None
        #You can also add another property to store how many leaves in under this node
        #self.size = 1
    def getStr(self):
        if self.left is None:
            leftStr = "()"
        else:
            leftStr = self.left.getStr()
        if self.right is None:
            rightStr = "()"
        else:
            rightStr = self.right.getStr()
        return "({}, {}, {}, {})".format(self.name, self.distance, leftStr, rightStr)
    def print(self):
        print(self.getStr())
    #for Q3
    def getChildrenSize(self): #only leaves node
        #return self.size
        if self.left is None and self.right is None:
            count = 1
        else:
            count = self.left.getChildrenSize() + self.right.getChildrenSize()
        return count

In [9]:
from upgmaData import *

In [10]:
#test for Q1
WX = TreeNode("W_X", 0.5)
WX.left = TreeNode("W", 0)
WX.right = TreeNode("X", 0)
WX.print()

(W_X, 0.5, (W, 0.0, (), ()), (X, 0.0, (), ()))


## 2. findClosestPair(Distances) [3pt]
Inputs Distances dictionary and returns the key for the pair of nodes that are closest. For
example, if the input is testDistances, the function returns (‘W’, ‘X’).

In [3]:
def findClosestPair(Distances):
    minValue = float("inf")
    minPair = None
    for node1, node2 in Distances:
        if node1 == node2:
            continue
        if Distances[node1, node2] < minValue:
            minValue = Distances[node1, node2]
            minPair = (node1, node2)
    return minPair

In [7]:
print(findClosestPair(testDistances))
print(findClosestPair(humansDistances))

('W', 'X')
('Finnish', 'Papuan')


## 3. updateDist(Distances, newNode) [3pt]
Inputs Distances dictionary and a new tree node. Updates the Distances dictionary by adding the
distances between newNode and all the other nodes. Does not calculate the distance between
newNode and its children in the Distances. This function returns the updated Distances
dictionary

In [11]:
def calcDistance(distances, node1, newNode):
    leftSize = newNode.left.getChildrenSize()
    rightSize = newNode.right.getChildrenSize()
    return (distances[node1, newNode.left] * leftSize + distances[node1, newNode.right] * rightSize) / (leftSize + rightSize)
    #You can also calculate this distance by two for loops adding distances for all possible pairs
    # between the two nodes (time-consuming)

def updateDist(distances, newNode):
    newDistances = {}
    for node1, node2 in distances:
        if (node1, newNode) not in newDistances:
            if node1 != newNode.left and node1 != newNode.right:
                newDistances[node1, newNode] = calcDistance(distances, node1, newNode)
                newDistances[newNode, node1] = newDistances[node1, newNode]
        if (node2, newNode) not in newDistances:
            if node2 != newNode.left and node2 != newNode.right:
                newDistances[node2, newNode] = calcDistance(distances, node2, newNode)
                newDistances[newNode, node2] = newDistances[node2, newNode]
        if node1 != newNode.left and node1 != newNode.right and node2 != newNode.left and node2 != newNode.right:
            newDistances[node1, node2] = distances[node1, node2]
    return newDistances
            

## 4. upgma(Distances) [4pt]
This function runs the UPGMA algorithm described in lecture. First, it initializes a list
TreeNodesList with multiple new TreeNode referring to humansList (the list TreeNodesList
should contain six TreeNodes). Then, it repeats the following steps until there is only one tree
node left in the list TreeNodesList, at which point this tree is returned.
<br>
a. Find the key for the closest pair of nodes in Distances (use findClosestPair)
<br>
b. Create a new TreeNode and assign the pair of nodes stored in TreeNodesList as its
children. Calculate the distance between the new TreeNode to its children and store the
value in the new TreeNode.
<br>
c. Update the TreeNodesList by removing the pair of nodes referring to a.
<br>
d. Update the Distances dictionary (use updateDist)
<br>
e. Add the new TreeNode into TreeNodesList

In [15]:
def merge(node1, node2, distance):
    newNode = TreeNode("[" + node1.name + "_" + node2.name + "]", distance / 2)
    
    #Optional to store distance to the root rather than to the bottom
    #node1.distance = newNode.distance - node1.distance
    #node2.distance = newNode.distance - node2.distance 

    newNode.left = node1
    newNode.right = node2
    #If you use size property, remember to update it.
    #newNode.size = node1.size + node2.size
    return newNode

def changeDistance(nodeList, distances):
    nodeDistances = {}
    for node1Name, node2Name in distances:
        nodeDistances[nodeList[node1Name], nodeList[node2Name]] = distances[node1Name, node2Name]
    return nodeDistances

def upgma(nameList, distances):
    nodes = {}
    for name in nameList:
        nodes[name] = TreeNode(name, 0)
    distances = changeDistance(nodes, distances)
    while len(nodes) != 1:
        node1, node2 = findClosestPair(distances)
        newNode = merge(node1, node2, distances[node1, node2])
        distances = updateDist(distances, newNode)
        nodes[newNode.name] = newNode
        del nodes[node1.name]
        del nodes[node2.name]
    return nodes.popitem()[1]

In [16]:
upgma(testList, testDistances).print()

([[W_X]_[Y_Z]], 1.25, ([W_X], 0.5, (W, 0.0, (), ()), (X, 0.0, (), ())), ([Y_Z], 0.5, (Y, 0.0, (), ()), (Z, 0.0, (), ())))


In [17]:
upgma(humansList, humansDistances).print()

([San_[Kikuyu_[Yoruba_[Han_[Finnish_Papuan]]]]], 0.0026804, (San, 0.0, (), ()), ([Kikuyu_[Yoruba_[Han_[Finnish_Papuan]]]], 0.0023785, (Kikuyu, 0.0, (), ()), ([Yoruba_[Han_[Finnish_Papuan]]], 0.0010831666666666665, (Yoruba, 0.0, (), ()), ([Han_[Finnish_Papuan]], 0.0009215, (Han, 0.0, (), ()), ([Finnish_Papuan], 0.000656, (Finnish, 0.0, (), ()), (Papuan, 0.0, (), ()))))))


In [131]:
#A more optimal way to design UPGMA
#I would also add size property to TreeNode class


import heapq
def calcDistance_heap(distances, node1, newNode):
    leftSize = newNode.left.getChildrenSize()
    rightSize = newNode.right.getChildrenSize()
    return (distances[node1.name, newNode.left.name] * leftSize + distances[node1.name, newNode.right.name] * rightSize) / (leftSize + rightSize)


def updateDistances_heap(nodes, distances, nodeDistances, newNode):
    for nodename in nodes:
        node = nodes[nodename]
        distances[nodename, newNode.name] = calcDistance_heap(distances, node, newNode)
        distances[newNode.name, nodename] = distances[nodename, newNode.name]
        heapq.heappush(nodeDistances, (distances[nodename, newNode.name], (nodename, newNode.name)))

def upgma_heap(nameList, distances):
    nodes = {}
    for name in nameList:
        nodes[name] = TreeNode(name, 0)
    nodeDistances = []
    for node1name, node2name in distances:
        if node1name != node2name:
            name1, name2 = sorted([node1name, node2name])
            if name1 == node1name and name2 == node2name:
                heapq.heappush(nodeDistances, (distances[name1, name2], (name1, name2)))
    usedNodes = set()
    while nodeDistances:
        minDistance, (node1name, node2name) = heapq.heappop(nodeDistances)
        if node1name in usedNodes or node2name in usedNodes:
            continue
        node1 = nodes[node1name]
        node2 = nodes[node2name]
        newNode = merge(node1, node2, distances[node1name, node2name])        
        del nodes[node1name]
        del nodes[node2name]
        updateDistances_heap(nodes, distances, nodeDistances, newNode)
        
        usedNodes.add(node1name)
        usedNodes.add(node2name)
        nodes[newNode.name] = newNode
    return newNode

In [132]:
upgma_heap(humansList, humansDistances).print()

([San_[Kikuyu_[Yoruba_[Han_[Finnish_Papuan]]]]], 0.0026804, (San, 0.0026804, (), ()), ([Kikuyu_[Yoruba_[Han_[Finnish_Papuan]]]], 0.0003018999999999999, (Kikuyu, 0.0023785, (), ()), ([Yoruba_[Han_[Finnish_Papuan]]], 0.0012953333333333334, (Yoruba, 0.0010831666666666665, (), ()), ([Han_[Finnish_Papuan]], 0.0001616666666666665, (Han, 0.0009215, (), ()), ([Finnish_Papuan], 0.0002655, (Finnish, 0.000656, (), ()), (Papuan, 0.000656, (), ()))))))
